In [ ]:
import tensorflow as tf
import os
os.environ['SM_FRAMEWORK'] = 'tf.keras'
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from segmentation_models_3D.losses import dice_loss
from segmentation_models_3D.metrics import iou_score

from GlottisNetV2.Utils.DataGenerator_3Dconv import DataGenerator
from GlottisNetV2.Utils.data import metric_mape3D, mape_ap, mape_pp, load_video_ap
from GlottisNetV2.Utils.Callbacks import get_callbacks
from GlottisNetV2.Models.GlottisNetV2e_3Dconv import glottisnetV2e_3Dconv

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Set path to training data
vid_training = r"Set path to training videos" # TODO

N_train = 640 # number of videos

# Create video IDs for training data and save them in Pandas Dataframe
cols = ['z', 'path']
df_imgs_train = pd.DataFrame(columns=cols)
df_segs_train = pd.DataFrame(columns=cols)

for i in tqdm(range(N_train)):
    vpath = vid_training + "\\" + str(i) + ".mp4"
    
    if os.path.exists(vpath):      
        new_img = {'z': i, 'path': vid_training + "/" + str(i) + ".mp4"}
        new_seg = {'z': i, 'path': vid_training + "/" + str(i) + "_mask.mp4"}

        df_imgs_train = df_imgs_train.append(new_img, ignore_index=True)
        df_segs_train = df_segs_train.append(new_seg, ignore_index=True)

print('Created IDs for training images.')

# Save coordinates of anterior and posterior points in Pandas Dataframe
cols = ['z', 'ap', 'pp']
training_data = pd.DataFrame(columns=cols)

for i in tqdm(range(N_train)):
    
    # Load video path
    path_vid = vid_training + "/" + str(i) + ".points"
    vpath = vid_training + "//" + str(i) + ".mp4"

    # If path exists, add video to dataframe
    if os.path.exists(vpath):
        ap_video = load_video_ap(path_vid, i)
        #training_data = pd.concat([training_data, ap_video], ignore_index=True)
        
        training_data = training_data.append(ap_video, ignore_index=True)
        

print('Loading of anterior and posterior points finished.')

In [ ]:
'''Training'''
# Set random seed for reproducible training
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
rand = np.random.seed(SEED)
random.seed(SEED)
tf.compat.v1.set_random_seed(SEED)

# Set parameters
FRAMES = 12
BATCH_SIZE = 8
FILTERS = 16
LAYERS = 4
LEARNING_RATE = 0.2e-3
EPOCHS = 100
TARGET_HEIGHT = 256
TARGET_WIDTH = 128
SHUFFLE = True
AUGMENT = True
MODEL_PATH = r"Set model path" # TODO
STEPS_PATH = r"Set path where model checkpoints are saved" # TODO
N_STEPS = 20 # Save every #N_STEPS epoch
RADIUS = 7.5

model = glottisnetV2e_3Dconv(input_size=(FRAMES, TARGET_HEIGHT, TARGET_WIDTH, 1), filters=FILTERS, layers=LAYERS)

# Hard split of training and validation data
train_vids, val_vids, train_segs, val_segs = train_test_split(df_imgs_train, df_segs_train,
                                                              test_size=0.1, random_state=SEED)

# Create new pandas dataframe for image ids
cols = ['z', 'path', 'id']
train_vids_new = pd.DataFrame(columns=cols)
val_vids_new = pd.DataFrame(columns=cols)
train_segs_new = pd.DataFrame(columns=cols)
val_segs_new = pd.DataFrame(columns=cols)

# Multiply number of videos for training and validation
for i in range(len(train_vids)):
    for j in range(10):           

        nr = random.randint(FRAMES, 29) 

        new_img = {'z': train_vids.iloc[i]['z'] , 'path': train_vids.iloc[i]['path'], 'id': nr}
        new_seg = {'z': train_segs.iloc[i]['z'] , 'path': train_segs.iloc[i]['path'], 'id': nr}

        train_vids_new  = train_vids_new.append(new_img, ignore_index=True)
        train_segs_new  = train_segs_new.append(new_seg, ignore_index=True)   
            
for i in range(len(val_vids)):
    for j in range(10):      

        nr= random.randint(FRAMES, 29)

        new_img = {'z': val_vids.iloc[i]['z'] , 'path': val_vids.iloc[i]['path'], 'id': nr}
        new_seg = {'z': val_segs.iloc[i]['z'] , 'path': val_segs.iloc[i]['path'], 'id': nr}

        val_vids_new  = val_vids_new.append(new_img, ignore_index=True)
        val_segs_new = val_segs_new.append(new_seg, ignore_index=True)
            
# Datagenerator --> Augmentation and Shuffle
# Training data
training_generator = DataGenerator(train_vids_new, train_segs_new, batch_size=BATCH_SIZE, target_height=TARGET_HEIGHT, 
                                   frame_nr=FRAMES, target_width=TARGET_WIDTH, shuffle=SHUFFLE, df_coordinates=training_data,
                                   augment=AUGMENT, radius=RADIUS)

# Validation data
validation_generator = DataGenerator(val_vids_new, val_segs_new, target_height=TARGET_HEIGHT, frame_nr=FRAMES, 
                                     target_width=TARGET_WIDTH, batch_size=BATCH_SIZE, shuffle=False,
                                     df_coordinates=training_data, augment=False, radius=RADIUS)

# Compile model with dice_loss for segmentation, mse for prediction maps and use Adam as optimizer
# First exit: predictions of anterior and posterior points (2 channels)
# Second exit: Segmentations
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              metrics={'seg': ['acc', iou_score],
                       'ap_pred': ['acc', metric_mape3D, mape_ap, mape_pp]},
              loss={'ap_pred': 'mse', 'seg': dice_loss},
              run_eagerly=True)

# Train model on training dataset and save it
model.fit(training_generator, validation_data=validation_generator, epochs=EPOCHS,
        callbacks=get_callbacks(MODEL_PATH, model, N_STEPS, STEPS_PATH))
